FST:

1) Use angsd for calculating saf files for each population

2) Use realSFS to calculate 2d sfs for each pair

3) Use the above calculated 2dsfs as priors jointly with all safs from step1 to calculate fst binary files

4) Use realSFS to extract the the fst values from the fst

So, I need the bam files sorted into populations and make a bam file list for each pop. Move them over to annotate using this

In [ ]:
 find . -name "*_pairedplus_marked_dupes.bam" -exec cp -i {} ../allBam/ \; 

In [ ]:
sed 's/$/_pairedplus_marked_dupes.bam/g' /local/home/sarahgw/WGS/Sequences/NJ_E/NJ_E_list>NJ_E_filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/NJ_E/ < /local/home/sarahgw/WGS/Sequences/NJ_E/NJ_E_filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/NJ_U/ < /local/home/sarahgw/WGS/Sequences/NJ_U/NJ_U_filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/VT_U/ < /local/home/sarahgw/WGS/Sequences/VT_U/VT_U_filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/NY_E/ < /local/home/sarahgw/WGS/Sequences/NY_E/NY_E.filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/NY_U/ < /local/home/sarahgw/WGS/Sequences/NY_U/NY_U.filelist

In [ ]:
xargs mv -t /local/home/sarahgw/WGS/Sequences/VT_E/ < /local/home/sarahgw/WGS/Sequences/VT_E/VT_E.filelist

Calculate SAF for each population

In [ ]:
~/angsd/angsd -P 10 -b NJ_E.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out NJ_E

In [ ]:
~/angsd/angsd -P 10 -b NJ_U.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out NJ_U

In [ ]:
~/angsd/angsd -P 10 -b VT_U.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out VT_U

In [ ]:
~/angsd/angsd -P 10 -b NY_U.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out NY_U

In [ ]:
~/angsd/angsd -P 10 -b NY_E.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out NY_E

In [ ]:
~/angsd/angsd -P 10 -b VT_E.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -GL 1 -dosaf 1 -minMapQ 30 -minQ 20 -out VT_E

run sfs for NJ samples

In [ ]:
~/angsd/misc/realSFS -P 15 -nsites 500000000 NJ_E.saf.idx NJ_U.saf.idx > NJ_EU

on annotate:

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 -nsites 500000000 NJ_E.saf.idx NJ_U.saf.idx > NJ_EU_sub

In [ ]:
~/angsd/angsd/misc/realSFS -P 15 NJ_E.saf.idx NJ_U.saf.idx > NJ_EU_full

Started at 4:43 PM on 4/6/17 on annotate

In [ ]:
~/angsd/misc/realSFS -P 15 -nsites 500000000 NY_E.saf.idx NY_U.saf.idx > NY_EU

Started at 11:30 AM 4/6/17 on amphiprion

In [ ]:
~/angsd/misc/realSFS -P 15 -nsites 500000000 VT_E.saf.idx VT_U.saf.idx > VT_EU

Started at 11:52 AM 4/6/17 on amphiprion. Killed too much memory?

Calculate fst

In [ ]:
~/angsd/misc/realSFS fst index NJ_E.saf.idx NJ_U.saf.idx -sfs NJ_EU.ml -fstout NJ_EU_fst

        -> args: tole:0.000001 nthreads:4 maxiter:100 nsites:500000000 start:NJ_EU.ml chr:(null) start:-1 stop:-1 fname:NJ_U.saf.idx fstout:NJ_EU_fst oldout:0 seed:1499364243 bootstrap:0 whichFst:0
        -> nSites: 500000000
        -> Reading: NJ_EU.ml assuming counts (will normalize to probs internally)
problem with size of dimension of prior 529 vs 2116

Try subsetting the .ml file using head to one line and rerunning? 

In [ ]:
~/angsd/misc/realSFS fst index NJ_E.saf.idx NJ_U.saf.idx -sfs NJ_EU.ml -fstout NJ_EU_fst -nsites 500000000